<h1>Import dataset:</h1>

In [6]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.spatial.distance import jaccard
from sklearn.preprocessing import Binarizer
from sklearn.metrics import jaccard_score

dfOriginal = pd.read_csv("funda.csv")
pd.set_option("display.max_columns", None)
print(dfOriginal)

          id                         address  area  bedrooms postal_code  \
0          0       Knollendamstraat 4 III/IV   108         2     1013 TN   
1          1             Moerdijkstraat 47 1    47         1     1079 XM   
2          2               Zeeburgerdijk 349   116         2     1095 AD   
3          3          Brederodestraat 124 -1    58         2     1054 VH   
4          4  Admiraal De Ruijterweg 409 III    63         1     1055 MD   
...      ...                             ...   ...       ...         ...   
11438  11438            Olympiaplein 83 -III   113         3     1077 CS   
11439  11439              Anjeliersstraat 20    88         2     1015 NH   
11440  11440        Rustenburgerstraat 389 I    77         2     1072 GV   
11441  11441         Doggersbankstraat 12 II    90         3     1055 NX   
11442  11442       Anna van Burenstraat 24 2    54         2     1055 VN   

      posting_date   price property_type  rooms  sale_date  \
0         4-6-2016  51000

<h1>Remove unnecessary columns:</h1>

In [7]:
df = dfOriginal

# Remove columns
df = df.drop("id", axis=1)
df = df.drop("posting_date", axis=1)
df = df.drop("sale_date", axis=1)
df = df.drop("price", axis=1)
df = df.drop("url", axis=1)
df = df.drop("postal_code", axis=1)
df = df.drop("address", axis=1)

df.property_type[df.property_type == 'apartment'] = 1
df.property_type[df.property_type == 'house'] = 0
df["property_type"] = df["property_type"].astype(int)

df

C:\Users\jvanw\AppData\Local\Temp\ipykernel_21356\2973974355.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.property_type[df.property_type == 'apartment'] = 1
C:\Users\jvanw\AppData\Local\Temp\ipykernel_21356\2973974355.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.property_type[df.property_type == 'house'] = 0


,area,bedrooms,property_type,rooms,year_built
0,108,2,1,3,1906
1,47,1,1,2,1938
2,116,2,1,3,2003
3,58,2,1,3,1910
4,63,1,1,3,1906
...,...,...,...,...,...
11438,113,3,1,5,1938
11439,88,2,1,3,1993
11440,77,2,1,4,1906
11441,90,3,1,4,1931


In [8]:
df.describe()

,area,bedrooms,property_type,rooms,year_built
count,11443.000000,11443.000000,11443.000000,11443.000000,11443.000000
mean,87.813860,2.180547,0.867867,3.369833,1947.916805
std,48.662899,1.079961,0.338650,1.392506,51.289763
min,1.000000,1.000000,0.000000,1.000000,1005.000000
25%,57.000000,1.000000,1.000000,2.000000,1906.000000
50%,77.000000,2.000000,1.000000,3.000000,1945.000000
75%,104.000000,3.000000,1.000000,4.000000,1990.000000
max,806.000000,12.000000,1.000000,19.000000,2017.000000


In [9]:
inputArea = int(input("Minimum amount of square metres of your temporary house:"))
inputBedrooms = int(input("Minimum amount of bedrooms to suit your family:"))
inputPropertyType = input("Would you like to live in an apartment or house?")
inputRooms = int(input("Besides bedrooms, how many regular rooms should your house minimally have to suit the needs of your family?"))
inputYear = int(input("Your house should preferably not be older than (insert year):"))


user_input = {}


user_input["area"] = inputArea
user_input["bedrooms"] = inputBedrooms
if inputPropertyType == "house":
    user_input["property_type"] = 1
elif inputPropertyType == "apartment":
    user_input["property_type"] = 0
user_input["rooms"] = inputRooms
user_input["year_built"] = inputYear


a = inputArea
b = inputBedrooms
d = inputPropertyType

if inputPropertyType == "house":
    d = 1
elif inputPropertyType == "apartment":
    d = 0

e = inputRooms
f = inputYear

user_input

{'area': 60, 'bedrooms': 2, 'property_type': 0, 'rooms': 2, 'year_built': 2000}

In [10]:
user_input_df = pd.DataFrame.from_dict(user_input, orient='index').T
user_input_df

,area,bedrooms,property_type,rooms,year_built
0,60,2,0,2,2000


In [11]:
user_input_df = pd.get_dummies(user_input_df, columns=["area", "bedrooms", "property_type", "rooms", "year_built"])

user_input_df.columns.values[0] = "area"
user_input_df.columns.values[1] = "bedrooms"
user_input_df.columns.values[2] = "property_type"
user_input_df.columns.values[3] = "rooms"
user_input_df.columns.values[4] = "year_built"

user_input_df

,area,bedrooms,property_type,rooms,year_built
0,1,1,1,1,1


In [12]:
binarizer_area = Binarizer(threshold=a)
df['area'] = binarizer_area.fit_transform(df['area'].values.reshape(-1, 1))

binarizer_bedrooms = Binarizer(threshold=b)
df['bedrooms'] = binarizer_bedrooms.fit_transform(df['bedrooms'].values.reshape(-1, 1))

binarizer_property = Binarizer(threshold=d)
df['property_type'] = binarizer_property.fit_transform(df['property_type'].values.reshape(-1, 1))

binarizer_rooms = Binarizer(threshold=e)
df['rooms'] = binarizer_rooms.fit_transform(df['rooms'].values.reshape(-1, 1))

binarizer_year = Binarizer(threshold=f)
df['year_built'] = binarizer_year.fit_transform(df['year_built'].values.reshape(-1, 1))


df

,area,bedrooms,property_type,rooms,year_built
0,1,0,1,1,0
1,0,0,1,0,0
2,1,0,1,1,1
3,0,0,1,1,0
4,1,0,1,1,0
...,...,...,...,...,...
11438,1,1,1,1,0
11439,1,0,1,1,0
11440,1,0,1,1,0
11441,1,1,1,1,0


In [13]:
jaccard_similarities = []

for idx, house in df.iterrows():
     jaccard_sim = jaccard_score([user_input_df['area'], user_input_df['bedrooms'], user_input_df['property_type'], user_input_df['rooms'], user_input_df['year_built']], [house['area'], house['bedrooms'], house['property_type'], house['rooms'], house['year_built']])
     
     jaccard_similarities.append(jaccard_sim)


df['jaccard_similarity'] = jaccard_similarities

df

,area,bedrooms,property_type,rooms,year_built,jaccard_similarity
0,1,0,1,1,0,0.6
1,0,0,1,0,0,0.2
2,1,0,1,1,1,0.8
3,0,0,1,1,0,0.4
4,1,0,1,1,0,0.6
...,...,...,...,...,...,...
11438,1,1,1,1,0,0.8
11439,1,0,1,1,0,0.6
11440,1,0,1,1,0,0.6
11441,1,1,1,1,0,0.8


In [14]:
# df.describe()

df = df.sort_values(by=['jaccard_similarity'], ascending=False)

df

,area,bedrooms,property_type,rooms,year_built,jaccard_similarity
8398,1,1,1,1,1,1.0
583,1,1,1,1,1,1.0
8228,1,1,1,1,1,1.0
565,1,1,1,1,1,1.0
6369,1,1,1,1,1,1.0
...,...,...,...,...,...,...
7668,0,0,0,0,0,0.0
4749,0,0,0,0,0,0.0
9455,0,0,0,0,0,0.0
9442,0,0,0,0,0,0.0


In [15]:
topThreeHouses = df.index[:3].tolist()

selected_rows = dfOriginal.loc[topThreeHouses]


selected_rows = selected_rows.drop("id", axis=1)
selected_rows = selected_rows.drop("area", axis=1)
selected_rows = selected_rows.drop("bedrooms", axis=1)
selected_rows = selected_rows.drop("posting_date", axis=1)
selected_rows = selected_rows.drop("price", axis=1)
selected_rows = selected_rows.drop("property_type", axis=1)
selected_rows = selected_rows.drop("rooms", axis=1)
selected_rows = selected_rows.drop("sale_date", axis=1)
selected_rows = selected_rows.drop("url", axis=1)
selected_rows = selected_rows.drop("year_built", axis=1)


# display the selected rows
print(selected_rows)

                          address postal_code
8398  Willem van Weldammelaan 125     1082 KT
583          Gaasterlandstraat 36     1079 RH
8228             Hoofdweg 708 +PP     1055 AB
